In [144]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [136]:
## go through all of them at once
#start at page 1


In [13]:
URL = "https://www2.hm.com/en_gb/men/shop-by-product/view-all.html"
URL = "https://www2.hm.com/en_gb/men/shop-by-product/view-all.html?page=1"

headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}

page = requests.get(URL, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [14]:
yes = soup.find("ul", class_="ed2eb5") # item that only is avaliable on valid pages
yes
if yes:
    print("yes")

yes


for each page

In [17]:

jobs  =soup.find_all( class_="c0c602" ) # find all items on page
len(jobs)

36

In [41]:
item_link = jobs[12].find("a")["href"]
print(item_link)
#jobs[2].find("img")["src"]

https://www2.hm.com/en_gb/productpage.0970818001.html


In [42]:
item_page = requests.get(item_link, headers=headers)
item_soup = BeautifulSoup(item_page.content, "html.parser")

In [45]:
#text
categories = item_soup.find("hm-breadcrumbs").find_all("a")
categories
print(item_link.split(".")[-2])
for category in categories[1:]:
    print(category.text)


0970818001
Men
Hoodies & Sweatshirts
Sweatshirts


In [44]:
# image

image = item_soup.find("div", class_="mini-slider").find("ul").children

for img in image:
    a = img.find("a")

    if type(a) != int:
        if a["aria-checked"] == "true":
            img_src_dirty = a.find("img")["src"]
            img_src = "http:"+img_src_dirty.replace("miniature", "fullscreen")
            print(img_src)

http://lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2F78%2F92%2F7892f6d0a9e9281545de3c119930b4e0e95691a3.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5B%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&call=url[file:/product/fullscreen]


## official

For Each item on a page:
open a product page

In [115]:

def get_item_data(item_link):
    item_data = []
    
    item_page = requests.get(item_link, headers=headers)
    item_soup = BeautifulSoup(item_page.content, "html.parser")
    
    article_id = item_link.split(".")[-2]
    item_data.append(article_id)
    #get categories
    categories = item_soup.find("hm-breadcrumbs").find_all("li")
    
    
    category_str = ""
    for category in categories[1:]:
        if category_str != "":
            category_str+= "/" 
        category_str+=category.text.replace("\n", "")
    item_data.append(category_str)
    
    # get imager link/data
    image = item_soup.find("div", class_="mini-slider").find("ul").children
    for img in image:
        a = img.find("a")
        # find the correct image that we want
        if type(a) != int: # check if it is not a int
            if a["aria-checked"] == "true": # check if the image is the one we want
                img_src_dirty = a.find("img")["src"]
                img_src = "http:"+img_src_dirty.replace("miniature", "fullscreen")
                item_data.append(img_src)
    #get link to item          
    item_data.append(item_link)
    
    return item_data

get_item_data('https://www2.hm.com/en_gb/productpage.1012056001.html')

['1012056001',
 'Relaxed Fit Sweatpants',
 'http://lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2Ffd%2F89%2Ffd899f78ad10cff9c646403adcd29615433d9639.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5B%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&call=url[file:/product/fullscreen]',
 'https://www2.hm.com/en_gb/productpage.1012056001.html']

In [119]:
jobs  =soup.find_all( class_="c0c602" ) # find all items on page

data = []
for job in jobs:
    item_link = job.find("a")["href"]
    data.append(get_item_data(item_link))
#data

[['0970819078',
  'Men/Hoodies & Sweatshirts/Hoodies/Loose Fit Hoodie',
  'http://lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2F9d%2F55%2F9d55d390f0a6b9f7162b9f5d0c4e49490a241662.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5Bmen_hoodiessweatshirts_hoodies%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&call=url[file:/product/fullscreen]',
  'https://www2.hm.com/en_gb/productpage.0970819078.html'],
 ['1002227001',
  'Men/Trousers/Cargo/Cargo joggers',
  'http://lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2Fb4%2F26%2Fb4261cae5350fe9209375bf73cf0014104d6e61f.jpg%5D%2Corigin%5Bdam%5D%2Ccategory%5B%5D%2Ctype%5BDESCRIPTIVESTILLLIFE%5D%2Cres%5Bm%5D%2Chmver%5B2%5D&call=url[file:/product/fullscreen]',
  'https://www2.hm.com/en_gb/productpage.1002227001.html'],
 ['0970819001',
  'Men/Hoodies & Sweatshirts/Hoodies/Loose Fit Hoodie',
  'http://lp2.hm.com/hmgoepprod?set=quality%5B79%5D%2Csource%5B%2F60%2F62%2F60629c6b2c3c690c247a6b1bff2d4541f4155d0c.jpg%5D%2Corigin%5Bdam

In [118]:
df = pd.DataFrame(data, columns=["article_id", "category", "image", "link"])
df

,article_id,category,image,link
0,0970819078,Men/Hoodies & Sweatshirts/Hoodies/Loose Fit Ho...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.09708190...
1,1002227001,Men/Trousers/Cargo/Cargo joggers,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.10022270...
2,0970819001,Men/Hoodies & Sweatshirts/Hoodies/Loose Fit Ho...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.09708190...
3,0945531001,Men/T-shirts & Tops/Multipacks/3-pack Regular ...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.09455310...
4,1106189001,Men/Trousers/Cargo/Regular Fit Ripstop cargo t...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.11061890...
5,1024256001,Men/Jeans/Slim /Slim Jeans,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.10242560...
6,1012056001,Relaxed Fit Sweatpants,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.10120560...
7,1064346001,Men/Trousers/Linen/Regular Fit Linen-blend tro...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.10643460...
8,0970819007,Men/Hoodies & Sweatshirts/Hoodies/Loose Fit Ho...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.09708190...
9,1019838001,Men/Trousers/Chinos/Chinos Slim/Slim Fit Cotto...,http://lp2.hm.com/hmgoepprod?set=quality%5B79%...,https://www2.hm.com/en_gb/productpage.10198380...


In [131]:

BASE_URL = "https://www2.hm.com/en_gb/men/shop-by-product/view-all.html?page="
page_no = 74


def fetch_page(URL, page):   
    headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}

    page = requests.get(URL+page, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")

    valid_page = soup.find("ul", class_="ed2eb5") # item that only is avaliable on valid pages
    return soup, valid_page


soup, valid_page = fetch_page(BASE_URL, str(page_no))

tot = 0
while valid_page:
    jobs  =soup.find_all( class_="c0c602" ) # find all items on page
    tot += len(jobs)
    print(page_no, tot)
    
    # get data
    
    page_no += 1
    soup, valid_page = fetch_page(BASE_URL, str(page_no))

yes
74 36
75 39


In [141]:
def get_page_item_links(page_no):
    soup, valid_page = fetch_page(BASE_URL, str(page_no))
    if valid_page:
        jobs  =soup.find_all( class_="c0c602" ) # find all items on page
        print(page_no, len(jobs))
        return len(jobs)
    else:
        return 0

In [142]:
get_items(0)

36

In [146]:
num_threads = 15
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit scraping tasks for each combination
    futures = [executor.submit(get_page_item_links, num) for num in range(1,75)]

    # Wait for all tasks to complete
    for future in futures:
        future.result()

8 36
7 36
6 36
9 36
13 36
3 36
14 36
4 36
15 36
10 36
5 36
2 36
1 36
11 36
12 36
16 36
1718 36
 36
19 36
22 36
2021 36
 36
23 36
26 36
25 36
27 36
24 36
30 36
28 36
29 36
31 36
33 36
34 36
35 36
32 36
38 36
37 36
40 36
36 36
39 36
41 36
45 36
44 36
43 36
46 36
42 36
50 36
47 36
49 36
48 36
51 36
52 36
54 36
53 36
56 36
55 36
57 36
59 36
58 36
60 36
61 36
65 36
62 36
64 36
63 36
67 36
66 36
71 36
69 36
74 36
68 36
73 36
70 36
72 36
